In [1]:
import torch
import sys
if '../src' not in sys.path:
    sys.path.append('../src')
from utils import *
from config import *
from retriever import Retriever
from chatbot import Chatbot

In [2]:
# Run on GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Running code @ {device}')

# Initialize logger
logger = get_logger("03_rag_pipeline")

wrapper = TextWrapper()
wrapper.width = 120
wrapper.replace_whitespace = False
wrapper.break_long_words = False
wrapper.break_on_hyphens = False

Running code @ cpu


### Document retriever

The document retriever is responsible for finding relevant documents based on the user question. It initializes an Opensearch client, uses a `neural` query for semantic search.

Use the `MODEL_ID` of the embedding model used at ingestion time to generate vector embeddings for the queries' text.

[OpenSearch Python Client docs](https://opensearch-project.github.io/opensearch-py/api-ref/clients/opensearch_client.html)

In [3]:
retriever = Retriever(logger=logger)

In [4]:
# query_text = "atributii responsabili de proces"
query_text = "bursa pentru studenti"

docs = retriever.retrieve_documents(query_text=query_text)

retriever.print_documents(docs)

{
    "page_content": "FINANȚAREA STUDIILOR (1) O persoană poate beneficia de finanțarea studiilor de la bugetul de stat, pentru un singur program de studiidin cadrulfiecărui ciclu de studii universitare (ciclul de scurtă durată, licență, masterat şi doctorat). (2) Un student poate fi înmatriculat concomitent la cel mult două programe în cadrul aceluiași ciclu de studii, indiferent de instituțiile de învățământ furnizoare de educație. Actele și documentele de studii se depun în original la facultatea / programul de studii unde studentul MINISTERUL EDUCAȚIEI Universitatea Naţională de Ştiinţă şi Tehnologie POLITEHNICA Bucureşti 9 beneficiază de finanțare de la buget, iar acolo unde este înmatriculat cu taxă, documentele se depun în copie conform cu originalul, însoțite de act doveditor de la instituția unde are depuse documentele în original. (3) POLITEHNICA București percepe taxe, în afara celor privind şcolarizarea suportate individual de către studenţi, şi în următoarele situaţii: a)

### RAG pipeline

Initialize llm

In [5]:
from langchain_openai import ChatOpenAI
import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

llm = ChatOpenAI(model="gpt-3.5-turbo")

In [6]:
chatbot = Chatbot(retriever=retriever, llm=llm)

In [7]:
query_text = "Cum pot obtine credite suplimentare?"
chatbot.run(query_text)
chatbot._print_conversation_history()

UTILIZATOR:
Cum pot obtine credite suplimentare?
------------------------------------------------------------------------------------------------------------------------
ASISTENT:
În cadrul Universității Politehnica București, conform Legii finanțelor publice nr. 500/2002, art. 22, ordonatorii de
credite răspund de angajarea, lichidarea și ordonanțarea cheltuielilor în limita creditelor bugetare repartizate și
aprobate. Pentru a obține credite suplimentare, trebuie să te adresezi ordonatorului de credite (conform procedurilor
interne stabilite în instituție) și să justifici necesitatea suplimentării creditului bugetar solicitat. Este important
să respecți normele și procedurile legale în vigoare pentru obținerea acestor credite suplimentare.

Surse:
-
Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autorizare-de-limitare-a-autoritatii-si-a-relatiilor-de-raportare-in-cadrul-DFC.pdf:
https://upb.ro/wp-content/uploads/2017/11/Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autoriz

In [8]:
chatbot.run("Cum pot obține bursă la facultate?")
chatbot._print_conversation_history()

UTILIZATOR:
Cum pot obtine credite suplimentare?
------------------------------------------------------------------------------------------------------------------------
ASISTENT:
În cadrul Universității Politehnica București, conform Legii finanțelor publice nr. 500/2002, art. 22, ordonatorii de
credite răspund de angajarea, lichidarea și ordonanțarea cheltuielilor în limita creditelor bugetare repartizate și
aprobate. Pentru a obține credite suplimentare, trebuie să te adresezi ordonatorului de credite (conform procedurilor
interne stabilite în instituție) și să justifici necesitatea suplimentării creditului bugetar solicitat. Este important
să respecți normele și procedurile legale în vigoare pentru obținerea acestor credite suplimentare.

Surse:
-
Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autorizare-de-limitare-a-autoritatii-si-a-relatiilor-de-raportare-in-cadrul-DFC.pdf:
https://upb.ro/wp-content/uploads/2017/11/Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autoriz

In [9]:
print(chatbot.state["conversation_text"])

Utilizator: Cum pot obtine credite suplimentare?
Asistent: În cadrul Universității Politehnica București, conform Legii finanțelor publice nr. 500/2002, art. 22, ordonatorii de credite răspund de angajarea, lichidarea și ordonanțarea cheltuielilor în limita creditelor bugetare repartizate și aprobate. Pentru a obține credite suplimentare, trebuie să te adresezi ordonatorului de credite (conform procedurilor interne stabilite în instituție) și să justifici necesitatea suplimentării creditului bugetar solicitat. Este important să respecți normele și procedurile legale în vigoare pentru obținerea acestor credite suplimentare.

Surse:
- Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autorizare-de-limitare-a-autoritatii-si-a-relatiilor-de-raportare-in-cadrul-DFC.pdf: https://upb.ro/wp-content/uploads/2017/11/Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autorizare-de-limitare-a-autoritatii-si-a-relatiilor-de-raportare-in-cadrul-DFC.pdf


In [10]:
chatbot.run("Ce valoare are?")
chatbot._print_conversation_history()

UTILIZATOR:
Cum pot obtine credite suplimentare?
------------------------------------------------------------------------------------------------------------------------
ASISTENT:
În cadrul Universității Politehnica București, conform Legii finanțelor publice nr. 500/2002, art. 22, ordonatorii de
credite răspund de angajarea, lichidarea și ordonanțarea cheltuielilor în limita creditelor bugetare repartizate și
aprobate. Pentru a obține credite suplimentare, trebuie să te adresezi ordonatorului de credite (conform procedurilor
interne stabilite în instituție) și să justifici necesitatea suplimentării creditului bugetar solicitat. Este important
să respecți normele și procedurile legale în vigoare pentru obținerea acestor credite suplimentare.

Surse:
-
Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autorizare-de-limitare-a-autoritatii-si-a-relatiilor-de-raportare-in-cadrul-DFC.pdf:
https://upb.ro/wp-content/uploads/2017/11/Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autoriz

In [11]:
print(chatbot.state["conversation_text"])

Utilizator: Cum pot obtine credite suplimentare?
Asistent: În cadrul Universității Politehnica București, conform Legii finanțelor publice nr. 500/2002, art. 22, ordonatorii de credite răspund de angajarea, lichidarea și ordonanțarea cheltuielilor în limita creditelor bugetare repartizate și aprobate. Pentru a obține credite suplimentare, trebuie să te adresezi ordonatorului de credite (conform procedurilor interne stabilite în instituție) și să justifici necesitatea suplimentării creditului bugetar solicitat. Este important să respecți normele și procedurile legale în vigoare pentru obținerea acestor credite suplimentare.

Surse:
- Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autorizare-de-limitare-a-autoritatii-si-a-relatiilor-de-raportare-in-cadrul-DFC.pdf: https://upb.ro/wp-content/uploads/2017/11/Procedura-de-desemnare-a-autoritatii-ierarhiilor-de-autorizare-de-limitare-a-autoritatii-si-a-relatiilor-de-raportare-in-cadrul-DFC.pdf
Utilizator: Cum pot obține bursă la facultat